In [42]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer

/home/kuper/projects/datich/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [43]:
df = pd.read_csv('./data/final/data_labelled.csv', engine='python')

In [44]:
dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.1)

In [ ]:
model_id = "Qwen/Qwen2.5-0.5B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["text"])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/681 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Map:   0%|          | 0/4415 [00:00<?, ? examples/s]

Map:   0%|          | 0/491 [00:00<?, ? examples/s]

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification
from peft import get_peft_model, LoraConfig, TaskType

model = AutoModelForSequenceClassification.from_pretrained(
    model_id,
    num_labels=6,
    problem_type="regression",
    device_map="auto",
    torch_dtype=torch.bfloat16
)
model.config.pad_token_id = tokenizer.pad_token_id

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/290 [00:00<?, ?it/s]

Qwen2ForSequenceClassification LOAD REPORT from: Qwen/Qwen2.5-0.5B
Key          | Status  | 
-------------+---------+-
score.weight | MISSING | 

Notes:
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


In [ ]:
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"],
    modules_to_save=["score"]
)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 1,086,720 || all params: 495,124,864 || trainable%: 0.2195


In [ ]:
from transformers import TrainingArguments, Trainer
import numpy as np
from sklearn.metrics import root_mean_squared_error

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.clip(predictions, 0.0, 1.0)
    rmse = root_mean_squared_error(labels, predictions)
    return {"rmse": rmse}

training_args = TrainingArguments(
    output_dir="./qwen_mental_health_scorer",
    eval_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=10,
    remove_unused_columns=False,
    bf16=True
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    processing_class=tokenizer, # <-- Update this line
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()
model.save_pretrained("./final_qwen_lora")
tokenizer.save_pretrained("./final_qwen_lora")

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.


Epoch,Training Loss,Validation Loss,Rmse
1,0.075000,0.082081,0.273136


Epoch,Training Loss,Validation Loss,Rmse
1,0.075000,0.082081,0.273136
2,0.048114,0.049394,0.219557
3,0.039461,0.044135,0.206591


('./final_qwen_lora/tokenizer_config.json',
 './final_qwen_lora/chat_template.jinja',
 './final_qwen_lora/tokenizer.json')

In [ ]:
import shutil
from google.colab import files

print("Zipping the model files...")
shutil.make_archive("final_qwen_lora", 'zip', "./final_qwen_lora")

print("Triggering download...")
files.download("final_qwen_lora.zip")

Zipping the model files...
Triggering download...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>